In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
"""
This is the random model for the wizard of wikipedia task. There is no trainable model here only random retrieval of utterances of wizards.
This model runs on the test dataset.

Deepak Goyal <deepak.16je002137@ece.iitism.ac.in>
"""

'\nThis is the random model for the wizard of wikipedia task. There is no trainable model here only random retrieval of utterances of wizards.\nThis model runs on the test dataset.\n\nDeepak Goyal <deepak.16je002137@ece.iitism.ac.in>\n'

In [0]:
import json
import os
import re
import math
import copy
import os
import string
import time
from sklearn.utils import shuffle
import random

In [0]:
# Specify which machine you are running things

running_device = 'colab'

In [0]:
def raw_data_loader(running_device, name):
    """ 
        Function for loading json data into a list
    """

    file_dir = ''
    file_name = name
    if running_device == 'local':
        file_dir = \
            '/home/naive/Documents/rohit/Wizard Of Wikipedia/Dataset'
    elif running_device == 'colab':
        file_dir = \
            '/content/drive/My Drive/Data/Wizard of Wikipedia/wizard_of_wikipedia'
    else:
        print('Invalid running device')
        return

    data = os.path.join(file_dir, file_name)

    json_data = None
    with open(data) as f:
        json_data = json.load(f)

    return json_data



In [10]:
json_data = None

print('Loading Raw Data into a list....')

t1 = time.time()
json_data_seen = raw_data_loader(running_device,
                                 'test_random_split.json')
json_data_unseen = raw_data_loader(running_device,
                                   'test_topic_split.json')
t2 = time.time()

print('Loading raw data took ' + str(t2 - t1) + ' seconds')


Loading Raw Data into a list....
Loading raw data took 10.206515312194824 seconds


In [11]:
print("Number of Conversations in seen test topics data: "+str(len(json_data_seen)))
print("Number of Conversations in unseen test topics data: "+str(len(json_data_unseen)))

Number of Conversations in seen test topics data: 965
Number of Conversations in unseen test topics data: 968


In [0]:
class vocabulary:

    """ Creates vocabulary for our corpus data of conversations"""

    def __init__(self):
        """
            self.word2Index(Dictionary): Maps words to tokens
            self.num_words(Scalar)     : Number of distinct words in the vocabulary
            
        """

        self.word2Index = {'PAD': 0, '<START>': 1, '<END>': 2}
        self.num_words = 3

    def addSentence(self, sentence):
        """ 
            Adds sentences into vocabulary by splitting them
        """

        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):

        if word not in self.word2Index.keys():
            self.word2Index[word] = self.num_words
            self.num_words += 1


In [0]:
# Preprocessing functions for the data


def remove_punctuation(sentence):

    # thanks to https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string

    return sentence.translate(str.maketrans('', '',
                              string.punctuation)).lower().strip()



In [14]:
def load_data(data_json):

    (context, knowledge, wizard) = ([], [], [])
    dict2cnt = {}
    count = 0

    N = len(data_json)
    for id in range(N):

        (current_knowledge, current_wizard, context_knowledge,
         previous_context) = ([], [], [], [])
        tmp_context = []

        current_conv = data_json[id]

        topic_chosen = remove_punctuation(current_conv['chosen_topic'])
        tmp_context.append(topic_chosen)

        conversation_length = len(current_conv['dialog'])

        for i in range(conversation_length):

            if current_conv['dialog'][i]['speaker'] == '0_Wizard':

                dict2cnt[count] = len(current_conv['dialog'
                        ][i]['retrieved_passages'])
                count = count + 1

                for x in range(len(current_conv['dialog'
                               ][i]['retrieved_passages'])):

                    idx = current_conv['dialog'][i]['retrieved_passages'
                            ][x]

                    for value in idx.values():

                        for j in range(len(value)):
                            value[j] = remove_punctuation(value[j])

                        current_knowledge.append(value)

                wizard_dialog = remove_punctuation(current_conv['dialog'
                        ][i]['text'])

                current_wizard.append(wizard_dialog)
                tmp_context.append(wizard_dialog)
            else:

                apperentice_dialog = \
                    remove_punctuation(current_conv['dialog'][i]['text'
                        ])
                tmp_context.append(apperentice_dialog)

        knowledge.append(current_knowledge)
        wizard.append(current_wizard)

        final_context = []
        flag = True

        if current_conv['dialog'][0]['speaker'] == '0_Wizard':
            flag = False

        for i in range(len(tmp_context)):
            temp = []

            for j in range(i + 1):
                temp.append(tmp_context[j])

            if flag:
                if i % 2 != 0 and len(final_context) \
                    < len(current_wizard):
                    final_context.append(temp)
            else:

                if i % 2 == 0 and len(final_context) \
                    < len(current_wizard):
                    final_context.append(temp)

        context.append(final_context)

    return (context, knowledge, wizard, dict2cnt)


print( "Getting test conversation's context, knowledge, wizard utterances..")
t1 = time.time()
(context, knowledge, wizard, dict2cnt) = load_data(json_data_seen)
(us_context, us_knowledge, us_wizard, us_dict2cnt) = \
    load_data(json_data_unseen)
t2 = time.time()
print('This took: ' + str(t2 - t1) + ' seconds')

Getting test conversation's context, knowledge, wizard utterances..
This took: 1.5522103309631348 seconds


In [15]:
num_wizard = 0
us_num_wizard = 0

for i in wizard:
    num_wizard += len(i)

for i in us_wizard:
    us_num_wizard += len(i)

print('Number of times wizard speaks in seen test dataset: ' \
    + str(num_wizard))
print('Number of times wizard speaks in unseen test dataset: ' \
    + str(us_num_wizard))


Number of times wizard speaks in seen test dataset: 2224
Number of times wizard speaks in unseen test dataset: 2075


In [16]:
def after_load_process(
    context,
    knowledge,
    wizard,
    dict2cnt,
    num,
    ):
    input_context = []
    input_knowledge = []
    output_wizard = []
    for i in range(len(wizard)):
        for c in context[i]:
            input_context.append(c)
        for k in knowledge[i]:
            input_knowledge.append(k)
        for w in wizard[i]:
            output_wizard.append(w)

    input_knowledge2 = []
    index = 0
    for i in range(num):
        r = dict2cnt[i]
        tmp = []
        fr = index
        to = index + r
        for j in range(fr, to):
            tmp.append(input_knowledge[index])
            index = index + 1
        input_knowledge2.append(tmp)
    input_knowledge3 = []
    for i in range(len(input_knowledge2)):
        g = []
        for j in range(len(input_knowledge2[i])):
            s = ''
            for k in range(len(input_knowledge2[i][j])):
                s = s + ' ' + input_knowledge2[i][j][k]
            s = s.strip()
            g.append(s)
        input_knowledge3.append(g)
    return (input_context, input_knowledge3, output_wizard)


print('Converting the extracted sentences into lists...')
t1 = time.time()
(final_context, final_knowledge, final_wizard) = \
    after_load_process(context, knowledge, wizard, dict2cnt, num_wizard)
(us_final_context, us_final_knowledge, us_final_wizard) = \
    after_load_process(us_context, us_knowledge, us_wizard,
                       us_dict2cnt, us_num_wizard)
t2 = time.time()
print('This took: ' + str(t2 - t1) + ' seconds')


Converting the extracted sentences into lists...
This took: 0.15073180198669434 seconds


In [0]:
max_length = 70


def process(sentence):
    """
        Preprocessing sentencing to make them of equal length and appending and terminating them with
        start (<START>) and end (<END>) token.
        
    """

    words = sentence.split()
    sentence_length = max_length - 2
    if len(words) <= sentence_length:
        for i in range(sentence_length - len(words)):
            words.append('PAD')
    else:
        words = words[:sentence_length]

    res = ''
    words.append('<END>')
    words.insert(0, '<START>')
    assert len(words) == max_length
    for w in words:
        res = res + ' ' + w
    res = res.strip()
    return res


In [18]:
vocab = vocabulary()


def process_loaded(input_context, input_knowledge, output_wizard):
    """ Processing sentences and forming vocabulary
    """

    for i in range(len(output_wizard)):
        output_wizard[i] = process(output_wizard[i])
        vocab.addSentence(output_wizard[i])
        assert len(output_wizard[i].split()) == max_length

    for i in range(len(input_knowledge)):
        for j in range(len(input_knowledge[i])):
            input_knowledge[i][j] = process(input_knowledge[i][j])
            vocab.addSentence(input_knowledge[i][j])
            assert len(input_knowledge[i][j].split()) == max_length

    for i in range(len(input_context)):
        for j in range(len(input_context[i])):
            input_context[i][j] = process(input_context[i][j])
            vocab.addSentence(input_context[i][j])
            assert len(input_context[i][j].split()) == max_length
    return (input_context, input_knowledge, output_wizard)


print('Processing fetched sentences...')
t1 = time.time()
(p_context, p_knowledge, p_wizard) = process_loaded(final_context,
        final_knowledge, final_wizard)
(us_p_context, us_p_knowledge, us_p_wizard) = \
    process_loaded(us_final_context, us_final_knowledge,
                   us_final_wizard)
t2 = time.time()
print('This process took: ' + str(t2 - t1))


Processing fetched sentences...
This process took: 3.3055410385131836


In [20]:
##########################SANITY CHECK##########################


def sanity_check(
    context,
    knowledge,
    wizard,
    num,
    data,
    ):
    assert len(wizard) == num
    assert len(context) == num
    assert len(knowledge) == num

    print('Running Sanity Checks on the data...' + data)

    t1 = time.time()

    for i in range(len(wizard)):
        assert len(wizard[i].split()) == max_length

    for i in range(len(context)):
        for j in range(len(context[i])):
            assert len(context[i][j].split()) == max_length

    for i in range(len(knowledge)):
        for j in range(len(knowledge[i])):
            assert len(knowledge[i][j].split()) == max_length


t1 = time.time()

sanity_check(p_context, p_knowledge, p_wizard, num_wizard,
             'test seen data')
sanity_check(us_p_context, us_p_knowledge, us_p_wizard, us_num_wizard,
             'test unseen data')

t2 = time.time()
print('Everything seems fine.')
print('Sanity Checks took: ' + str(t2 - t1) + ' seconds')

################################################################


Running Sanity Checks on the data...test seen data
Running Sanity Checks on the data...test unseen data
Everything seems fine.
Sanity Checks took: 0.2047266960144043 seconds


In [0]:
########################Shuffling data########################

(p_context, p_knowledge, p_wizard) = shuffle(p_context, p_knowledge,
        p_wizard)
(us_p_context, us_p_knowledge, us_p_wizard) = shuffle(us_p_context,
        us_p_knowledge, us_p_wizard)


In [0]:
def score_F1(true, predicted):
    tokenized_predicted = [vocab.word2Index[t] for t in
                           predicted.split()]
    tokenized_true = [vocab.word2Index[t] for t in true.split()]

    length = 0
    pos = 0

    for i in tokenized_predicted:
        if i == 0 or i == 1 or i == 2:
            continue
        length = length + 1
        if i in tokenized_true:
            pos = pos + 1

    return pos / length


In [0]:
def testing(wizard, num):

    positive = 0.0
    F1 = 0.0

    for i in range(len(wizard)):

#         thanks https://stackoverflow.com/questions/3996904/generate-random-integers-between-0-and-9

        x = random.randrange(num)

        if x == i:
            positive = positive + 1

        F1 += score_F1(wizard[i], wizard[x])

    positive = positive / num
    F1 = F1 / num * 100
    return (positive, F1)


In [24]:
avg_r1 = 0
avg_f1 = 0
for i in range(num_wizard):
    (r1, f1) = testing(p_wizard, num_wizard)
    avg_r1 += r1
    avg_f1 += f1

print('The Average R@1 on the test seen data is: ' + str(avg_r1
        / num_wizard))
print('The Average F1 on the test seen data is: ' + str(avg_f1
        / num_wizard))

The Average R@1 on the test seen data is: 0.00045530122664459385
The Average F1 on the test seen data is: 10.775657375340845


In [25]:
avg_r1 = 0
avg_f1 = 0

for i in range(us_num_wizard):
    (r1, f1) = testing(us_p_wizard, us_num_wizard)
    avg_r1 += r1
    avg_f1 += f1

print('The Average R@1 on test unseen data is: ' + str(avg_r1
        / us_num_wizard))
print('The Average F1 on test unseen data is: ' + str(avg_f1
        / us_num_wizard))


The Average R@1 on test unseen data is: 0.00047240528378573635
The Average F1 on test unseen data is: 10.941896969480037
